In [9]:
import pandas as pd

csv_path = "../experiment_results/Variance_Decoherence/results.csv"
df = pd.read_csv(csv_path)

df["variance_level"] = df["backend"].str.extract(r"variance_(none|low|mid|high)")
df["backend_suffix"] = df["backend"].str.extract(r"_(\d+)$")

df = df[df["backend_suffix"].isin(["1", "10"])]

variance_order = ["none", "low", "mid", "high"]
df["variance_level"] = pd.Categorical(df["variance_level"], categories=variance_order, ordered=True)

df = df.sort_values(by=["backend_suffix", "code", "variance_level"])

# Percentage change between consecutive variance levels
df["percent_change"] = (
    df.groupby(["backend_suffix", "code"])["logical_error_rate"]
      .pct_change() * 100
)

# Magnitude change (absolute difference) between consecutive variance levels
df["magnitude_change"] = (
    df.groupby(["backend_suffix", "code"])["logical_error_rate"]
      .diff()
)

summary = (
    df[["backend_suffix", "code", "variance_level",
        "logical_error_rate", "percent_change", "magnitude_change"]]
    .reset_index(drop=True)
)

pivot = summary.pivot_table(
    index=["backend_suffix", "code"],
    columns="variance_level",
    values="logical_error_rate"
)

print("=== Per-step percentage change (none → low → mid → high) ===")
display(summary)

print("\n=== Logical error rates by variance level ===")
display(pivot.round(3))


=== Per-step percentage change (none → low → mid → high) ===


/tmp/ipykernel_36435/657163190.py:34: FutureWarning: The default value of observed=False is deprecated and will change to observed=True in a future version of pandas. Specify observed=False to silence this warning and retain the current behavior
  pivot = summary.pivot_table(


,backend_suffix,code,variance_level,logical_error_rate,percent_change,magnitude_change
0,1,bacon,none,0.492,NaN,NaN
1,1,bacon,low,0.485,-1.422764,-0.007
2,1,bacon,mid,0.500,3.092784,0.015
3,1,bacon,high,0.509,1.800000,0.009
4,1,color,none,0.503,NaN,NaN
5,1,color,low,0.502,-0.198807,-0.001
6,1,color,mid,0.494,-1.593625,-0.008
7,1,color,high,0.534,8.097166,0.040
8,1,gross,none,1.000,NaN,NaN
9,1,gross,low,1.000,0.000000,0.000



=== Logical error rates by variance level ===


variance_level           none    low    mid   high
backend_suffix code                               
1              bacon    0.492  0.485  0.500  0.509
               color    0.503  0.502  0.494  0.534
               gross    1.000  1.000  0.999  1.000
               hh       0.497  0.504  0.500  0.497
               steane   0.300  0.320  0.317  0.328
               surface  0.493  0.498  0.489  0.489
10             bacon    0.488  0.512  0.512  0.505
               color    0.290  0.341  0.315  0.222
               gross    0.301  0.435  0.377  0.376
               hh       0.065  0.086  0.064  0.038
               steane   0.105  0.086  0.074  0.080
               surface  0.050  0.070  0.056  0.017

In [8]:
import pandas as pd

# ---- Paths to CSVs ----
deco_csv = "../experiment_results/Variance_Decoherence/results.csv"
readout_csv = "../experiment_results/Variance_Readout/results.csv"

def compute_avg_magnitude_change(csv_path):
    df = pd.read_csv(csv_path)

    # Extract variance level and backend suffix
    df["variance_level"] = df["backend"].str.extract(r"variance_(none|low|mid|high)")
    df["backend_suffix"] = df["backend"].str.extract(r"_(\d+)$")

    df = df[df["backend_suffix"].isin(["1", "10"])]
    
    variance_order = ["none", "low", "mid", "high"]
    df["variance_level"] = pd.Categorical(df["variance_level"], categories=variance_order, ordered=True)

    # Sort to compute differences
    df = df.sort_values(by=["backend_suffix", "code", "variance_level"])

    # Compute magnitude change (absolute difference between consecutive variance levels)
    df["magnitude_change"] = df.groupby(["backend_suffix", "code"])["logical_error_rate"].diff().abs()

    # Average over all codes and suffixes
    avg_magnitude_change = df["magnitude_change"].mean()

    return avg_magnitude_change

# ---- Compute average magnitude changes ----
deco_avg_mag = compute_avg_magnitude_change(deco_csv)
read_avg_mag = compute_avg_magnitude_change(readout_csv)

print(f"Average magnitude change - Decoherence: {deco_avg_mag:.3f}")
print(f"Average magnitude change - Readout:     {read_avg_mag:.3f}")


Average magnitude change - Decoherence: 0.020
Average magnitude change - Readout:     0.021


In [10]:
import pandas as pd

df = pd.read_csv("../experiment_results/Decoder/results.csv")

# Columns to average
metrics = [
    "logical_error_rate",
    "time_decode",
    "mem_decode_full_MB",
]

# Exclude Gross code (case-insensitive, robust)
df_filtered = df[~df["code"].str.lower().eq("gross")]

# Group by decoder and compute averages
decoder_averages = (
    df_filtered
    .groupby("decoder")[metrics]
    .mean()
    .reset_index()
)

decoder_averages


,decoder,logical_error_rate,time_decode,mem_decode_full_MB
0,bposd_batch,0.1772,29554.168736,1079.945626
1,bposd_chk,0.1676,30706.357405,1091.887104
2,mwpm,0.3142,0.342717,798.244045


In [2]:
import pandas as pd

# --- CSVs ---
csv_common = "../experiment_results/Program_stats_topology/results.csv"
csv_big = "../experiment_results/Program_stats_size/results.csv"
csv_more_shots = "../experiment_results/Program_stats_shots/results.csv"

# --- Load data ---
df_common = pd.read_csv(csv_common)
df_big = pd.read_csv(csv_big)
df_more = pd.read_csv(csv_more_shots)

# --- Select relevant rows ---
row_baseline = df_common[df_common["backend"].str.contains("full", case=False)].iloc[0]
row_grid = df_common[df_common["backend"].str.contains("grid", case=False)].iloc[0]
row_big = df_big.iloc[0]
row_more = df_more.iloc[0]

cases = [
    ("Baseline", row_baseline),
    ("Bigger Backend", row_big),
    ("Sparser Backend", row_grid),
    ("More Shots", row_more),
]

# --- Compute decoding percentages ---
decoding_percentages = {}
for name, row in cases:
    total_time = row["total_time"]
    decode_time = row["time_decode"]
    percent = (decode_time / total_time) * 100
    decoding_percentages[name] = percent

# Print each case
print("Decoding time as % of total runtime:")
for name, pct in decoding_percentages.items():
    print(f"{name}: {pct:.2f}%")

# --- Average decoding percentage ---
average_pct = sum(decoding_percentages.values()) / len(decoding_percentages)
print(f"\nAverage decoding time across all cases: {average_pct:.2f}%")


Decoding time as % of total runtime:
Baseline: 99.91%
Bigger Backend: 99.93%
Sparser Backend: 99.99%
More Shots: 99.99%

Average decoding time across all cases: 99.96%


In [9]:
import pandas as pd

csv_path = "../experiment_results/Accumulated_Small/results.csv" 
df = pd.read_csv(csv_path)

high_corrected = df[df["corrected_error_rate"] > df["error_probability"]]

# Keep only the row with smallest error_probability per code
smallest_per_code = high_corrected.loc[high_corrected.groupby("code")["error_probability"].idxmin()]

print("Smallest error probability per code where corrected_error_rate > physical error_probability:")
print(smallest_per_code[["code", "error_probability", "corrected_error_rate"]])
nonzero_idle = df[df["idle_error_rate"] > 0]
print("\nRows with non-zero idle_error_rate:")
print(nonzero_idle)

raw_le_corrected = df[df["raw_error_rate"] <= df["corrected_error_rate"]].sort_values(["code", "error_probability"])
print("\nRows where raw_error_rate <= corrected_error_rate:")
print(raw_le_corrected[["code", "error_probability", "raw_error_rate", "corrected_error_rate"]])

Smallest error probability per code where corrected_error_rate > physical error_probability:
       code  error_probability  corrected_error_rate
13    bacon              0.001                 0.025
15    color              0.002                 0.003
18    gross              0.004                 0.031
10       hh              0.002                 0.008
2    steane              0.001                 0.046
24  surface              0.004                 0.007

Rows with non-zero idle_error_rate:
Empty DataFrame
Columns: [backend, backend_size, code, decoder, distance, cycles, num_samples, error_type, error_probability, raw_error_rate, idle_error_rate, corrected_error_rate]
Index: []

Rows where raw_error_rate <= corrected_error_rate:
      code  error_probability  raw_error_rate  corrected_error_rate
30   bacon              0.004           0.488                 0.491
32   bacon              0.006           0.507                 0.509
31   bacon              0.008           0.507       